In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys, MufexTestKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.indicators.tv_indicators import *
from quantfreedom.plotting.plotting_base import *
from quantfreedom.exchanges.exchange import Exchange
from quantfreedom.helper_funcs import dl_ex_candles, candles_to_df
from my_stuff import BybitTestKeys
from quantfreedom.exchanges.bybit_exchange.bybit import Bybit


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [ ]:
candles = dl_ex_candles(exchange="binance_us", symbol="BTCUSDT", timeframe="1h", candles_to_dl=200)
open = candles[:, 1]
high = candles[:, 2]
low = candles[:, 3]
close = candles[:, 4]
close_shift = np.roll(candles[:, 4], 1)

In [ ]:
candles_to_df(Bybit(use_test_net=False).get_candles(symbol="BTCUSDT", timeframe="1h", candles_to_dl=26))

In [3]:
bybit_test = Bybit(api_key=BybitTestKeys.api_key, secret_key=BybitTestKeys.secret_key, use_test_net=True)

In [18]:
bybit_test.create_order(
    category="linear",
    symbol="BTCUSDT",
    buy_sell="Buy",
    order_type="Market",
    asset_size=0.1,
    position_mode=1,
)

'46057ffe-cd4c-4263-8fa0-677d4c35d4a3'